In [ ]:
%pip install -q ipywidgets

In [ ]:
import json
import random
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

In [21]:
articles_file = "DiversiNews/NewsSpectrum/articles.json"
with open(articles_file, "r", encoding="utf-8") as f:
    articles = json.load(f)
    
title = widgets.HTML(value="<h1>Recreation of DiversiNews</h1>")
subtitle = widgets.HTML(value='<h3>Paper: <a href="https://dl.acm.org/doi/abs/10.14778/3685800.3685854" target="_blank">Link to Paper</a></h3>')

heading = widgets.VBox([title, subtitle])

for article in articles:
    if "sentiment" not in article:
        article["sentiment"] = random.uniform(-1, 1)

label_widget = widgets.HTML(value="<h4>Prioritize news with sentiment that is:</h4>")

def get_sentiment_descriptor(score):
    if score < -0.5:
        return "very negative"
    elif score < 0:
        return "negative"
    elif score < 0.5:
        return "positive"
    else:
        return "very positive"

sentiment_slider = widgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.1,
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

left_label = widgets.Label(value="negative", layout=widgets.Layout(width='60px'))
right_label = widgets.Label(value="positive", layout=widgets.Layout(width='60px'))

slider_with_labels = widgets.HBox([left_label, sentiment_slider, right_label])

spacer = widgets.HTML(value="<div style='margin-top: 20px;'></div>")

# sentiment_descriptor_label = widgets.Label(value=get_sentiment_descriptor(sentiment_slider.value))
widgets.HTML(
    value=f"<b>{get_sentiment_descriptor(sentiment_slider.value)}</b>"
)

articles_output = widgets.Output()

def update_articles(change):
    articles_output.clear_output()
    
    current_sentiment = sentiment_slider.value
    
    # sentiment_descriptor_label.value = get_sentiment_descriptor(current_sentiment)
    
    sorted_articles = sorted(articles, key=lambda a: abs(a["sentiment"] - current_sentiment))
    top_articles = sorted_articles[:10]
    
    with articles_output:
        for article in top_articles:
            display(HTML(
                f"<div style='padding-bottom:20px;'>"
                f"  <div><strong>{article['title']}</strong></div>"
                f"  <div><a href='{article['_id']}' target='_blank'>{article['_id']}</a></div>"
                f"  <div>{article['excerpt']}</div>"
                f"</div>"
            ))

sentiment_slider.observe(update_articles, names='value')

update_articles({'new': sentiment_slider.value})

ui = widgets.VBox([heading, 
                   label_widget, 
                   slider_with_labels,
                   spacer,
                #    sentiment_descriptor_label, 
                   articles_output])
display(ui)